## Building an ETL Data Pipeline
  Step 1: Extract data from the CSV file into a Pandas Dataframe
  Step 2: Transform the data(i.e clean the data) - Deal with missing and duplicate data
  Step 3: Create a database
  Step 4: Load the clean data into the database

Press SHIFT + Enter (Press together) - To Run you Code

In [10]:
# Import Libraries

import pandas as pd # For data Extract/transformation/manipulation/wrangling/analysis, etc
import psycopg2 # For Connecting Python to Postgresql database
from sqlalchemy import create_engine # To efficiently manage and reuse the database connections

### Step 1: Extract data from the Excel file into a Pandas Dataframe

In [11]:
# read the excel file
data = pd.read_excel("Employee.xlsx")

# view top five
data.head()

,Unnamed: 0,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,...,1.0,6,1,0,0,3.0,4.0,2.0,3,3
1,1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Female,...,6.0,3,5,1,4,3.0,2.0,4.0,2,4
2,2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,Male,...,5.0,2,5,0,3,2.0,2.0,1.0,3,3
3,3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,Male,...,13.0,5,8,7,5,4.0,4.0,3.0,2,3
4,4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,Male,...,9.0,2,6,0,4,4.0,1.0,3.0,3,3


In [12]:
# view last five
data.tail()

,Unnamed: 0,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
4411,11,36,No,Travel_Rarely,Research & Development,28,1,Life Sciences,1,Male,...,16.0,2,15,10,11,NaN,4.0,3.0,3,3
4412,12,55,No,Travel_Rarely,Research & Development,14,4,Life Sciences,1,Female,...,37.0,2,36,4,13,4.0,1.0,3.0,3,3
4413,13,47,Yes,Non-Travel,Research & Development,1,1,Medical,1,Male,...,10.0,4,10,9,9,1.0,2.0,2.0,2,3
4414,14,28,No,Travel_Rarely,Research & Development,1,3,Life Sciences,1,Male,...,5.0,2,5,0,4,4.0,4.0,2.0,3,3
4415,15,37,No,Travel_Rarely,Research & Development,1,3,Life Sciences,1,Male,...,7.0,2,5,0,1,3.0,4.0,4.0,3,3


### Step 2: Transform the data(i.e clean the data) - Deal with missing and duplicate data

In [13]:
data.duplicated().sum()

np.int64(6)

In [14]:
# remove duplicate record - keep first or last 
data.drop_duplicates(keep="first", inplace=True)

In [15]:
data.duplicated().sum()

np.int64(0)

In [16]:
# Explore missing data
data.isnull().sum()

Unnamed: 0                  0
Age                         0
Attrition                   0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              3
EmployeeCount               0
Gender                      0
JobLevel                    0
JobRole                     0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked         19
Over18                      0
PercentSalaryHike           0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
EnvironmentSatisfaction    25
JobSatisfaction            20
WorkLifeBalance            38
JobInvolvement              0
PerformanceRating           0
dtype: int64

In [17]:
# let's look at missing data - EducationField 
data[data['EducationField'].isnull()].head()

,Unnamed: 0,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
5,5,46,No,Travel_Rarely,Research & Development,8,3,NaN,1,Female,...,28.0,5,7,7,7,3.0,2.0,2.0,3,3
20,20,26,No,Travel_Frequently,Research & Development,1,4,NaN,1,Male,...,6.0,3,6,1,4,3.0,2.0,1.0,3,3
22,22,53,No,Travel_Rarely,Research & Development,11,4,NaN,1,Female,...,21.0,2,5,1,3,3.0,3.0,2.0,3,3


In [18]:
#deal with missin data 
data ['EducationField'].fillna('unknown', inplace=True)

C:\Users\imash\AppData\Local\Temp\ipykernel_36736\1422851914.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data ['EducationField'].fillna('unknown', inplace=True)


In [19]:
# let's look at missing data - NumCompaniesWorked
data[data['NumCompaniesWorked'].isnull()].head()

,Unnamed: 0,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
115,115,27,No,Travel_Rarely,Sales,2,3,Life Sciences,1,Male,...,5.0,2,5,0,4,4.0,2.0,3.0,3,3
210,210,52,No,Travel_Rarely,Sales,2,1,Medical,1,Male,...,18.0,2,10,6,9,4.0,1.0,3.0,3,3
343,343,40,No,Travel_Frequently,Sales,9,3,Marketing,1,Male,...,10.0,3,1,0,0,1.0,3.0,3.0,3,3
476,476,22,No,Travel_Rarely,Research & Development,8,3,Other,1,Male,...,4.0,2,4,1,1,2.0,1.0,2.0,3,4
647,647,29,No,Travel_Rarely,Research & Development,3,3,Technical Degree,1,Female,...,9.0,2,7,1,7,4.0,4.0,2.0,3,3


In [26]:
#deal with missin data 
data['NumCompaniesWorked'] = data['NumCompaniesWorked'].fillna(0)
data['TotalWorkingYears'] = data['TotalWorkingYears'].fillna(0)
data['EnvironmentSatisfaction'] = data['EnvironmentSatisfaction'].fillna(0)
data['JobSatisfaction'] = data['JobSatisfaction'].fillna(0)
data['WorkLifeBalance'] = data['WorkLifeBalance'].fillna(0)



In [28]:
# Explore missing data
data.isnull().sum()

Unnamed: 0                 0
Age                        0
Attrition                  0
BusinessTravel             0
Department                 0
DistanceFromHome           0
Education                  0
EducationField             0
EmployeeCount              0
Gender                     0
JobLevel                   0
JobRole                    0
MaritalStatus              0
MonthlyIncome              0
NumCompaniesWorked         0
Over18                     0
PercentSalaryHike          0
StandardHours              0
StockOptionLevel           0
TotalWorkingYears          0
TrainingTimesLastYear      0
YearsAtCompany             0
YearsSinceLastPromotion    0
YearsWithCurrManager       0
EnvironmentSatisfaction    0
JobSatisfaction            0
WorkLifeBalance            0
JobInvolvement             0
PerformanceRating          0
dtype: int64

In [25]:
# Save the modified DataFrame back to the original Excel file
data.to_excel('Employee.xlsx', index=False)

### Step 3: Create a database
  go to PGAdmin 4 and create database table

### Step 4: Load the clean data into the database

In [50]:
# Database credentials
username = 'postgres'
password = 'imashi79'
host = 'localhost'
port = 5432
db_name = 'ELT_warehouse'

In [54]:
# Establish a connection
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{db_name}')
    

In [56]:
# load the database table - Employee
data.to_sql('Employee', engine, if_exists='replace', index=False)

#close the connection
engine.dispose()